# Operations
elementwise operations:scalar operation, some standard binary operator on each pair of corresponding elements from the two arrays, any function that maps from a scalar to a scalar.

In [1]:
%%time
import torch

Wall time: 1.32 s


The common standard arithmetic operators (+, -, *, /, and **) have all been lifted to elementwise operations for any identically-shaped tensors of arbitrary shape.

In [2]:
x = torch.tensor([1.0, 2, 4, 8])
y = torch.tensor([2, 2, 2, 2])
x + y, x - y, x * y, x / y, x ** y  # The ** operator is exponentiation


(tensor([ 3.,  4.,  6., 10.]),
 tensor([-1.,  0.,  2.,  6.]),
 tensor([ 2.,  4.,  8., 16.]),
 tensor([0.5000, 1.0000, 2.0000, 4.0000]),
 tensor([ 1.,  4., 16., 64.]))

In [3]:
z = torch.tensor([3, 4, 5, 6, 7])
z + y, z - y, z * y, z / y, z ** y  # The ** operator is ezponentiation

RuntimeError: The size of tensor a (5) must match the size of tensor b (4) at non-singleton dimension 0

In [4]:
torch.exp(x)

tensor([2.7183e+00, 7.3891e+00, 5.4598e+01, 2.9810e+03])

In [5]:
x = torch.arange(12, dtype=torch.float32).reshape((3,4))
y = torch.tensor([[2.0, 1, 4, 3], [1, 2, 3, 4], [4, 3, 2, 1]])
torch.cat((x, y), dim=0), torch.cat((x, y), dim=1)

(tensor([[ 0.,  1.,  2.,  3.],
         [ 4.,  5.,  6.,  7.],
         [ 8.,  9., 10., 11.],
         [ 2.,  1.,  4.,  3.],
         [ 1.,  2.,  3.,  4.],
         [ 4.,  3.,  2.,  1.]]),
 tensor([[ 0.,  1.,  2.,  3.,  2.,  1.,  4.,  3.],
         [ 4.,  5.,  6.,  7.,  1.,  2.,  3.,  4.],
         [ 8.,  9., 10., 11.,  4.,  3.,  2.,  1.]]))

construct a binary ndarray via logical statements. 

In [6]:
x == y

tensor([[False,  True, False,  True],
        [False, False, False, False],
        [False, False, False, False]])

In [7]:
x < y

tensor([[ True, False,  True, False],
        [False, False, False, False],
        [False, False, False, False]])

In [8]:
x > y

tensor([[False, False, False, False],
        [ True,  True,  True,  True],
        [ True,  True,  True,  True]])

In [9]:
x.sum()

tensor(66.)

In [10]:
a = torch.arange(3).reshape((3, 1))
b = torch.arange(2).reshape((1, 2))
a, b

(tensor([[0],
         [1],
         [2]]),
 tensor([[0, 1]]))

Since a and b are  3×1  and  1×2  matrices respectively, their shapes do not match up if we want to add them. We broadcast the entries of both matrices into a larger  3×2  matrix as follows: for matrix a it replicates the columns and for matrix b it replicates the rows before adding up both elementwise.

In [11]:
a + b

tensor([[0, 1],
        [1, 2],
        [2, 3]])

[-1] selects the last element and [1:3] selects the second and the third elements as follows:

In [12]:
x[-1], x[1:3]

(tensor([ 8.,  9., 10., 11.]),
 tensor([[ 4.,  5.,  6.,  7.],
         [ 8.,  9., 10., 11.]]))

In [13]:
x[1, 2] = 9
x

tensor([[ 0.,  1.,  2.,  3.],
        [ 4.,  5.,  9.,  7.],
        [ 8.,  9., 10., 11.]])

In [14]:
x [0]

tensor([0., 1., 2., 3.])

In [15]:
x[0:2, :] = 12
x

tensor([[12., 12., 12., 12.],
        [12., 12., 12., 12.],
        [ 8.,  9., 10., 11.]])

#### Running operations can cause new memory to be allocated to host results.
For example, if we write y = x + y, we will dereference the ndarray that y used to point to and instead point y at the newly allocated memory.

In [16]:
before = id(y)
y = y + x
id(y) == before

False

In [17]:
id(x) < before
id(x) > before

True

This might be undesirable for two reasons. 

1. we do not want to run around allocating memory unnecessarily all the time. In machine learning, we might have hundreds of megabytes of parameters and update all of them multiple times per second. Typically, we will want to perform these updates in place. 

2. we might point at the same parameters from multiple variables. If we do not update in place, other references will still point to the old memory location, making it possible for parts of our code to inadvertently reference stale parameters.

We can assign the result of an operation to a previously allocated array with slice notation, e.g., y[:] = <expression>.

In [18]:
z = torch.zeros_like(y)
print('id(z):', id(z))
z[:] = x + y
print('id(z):', id(z))

id(z): 2584353511608
id(z): 2584353511608


If the value of x is not reused in subsequent computations, we can also use x[:] = x + y or x += y to reduce the memory overhead of the operation.

In [19]:
before = id(x)
x += y
id(x) == before

True

In [20]:
id(x) < before
id(x) > before

False

In [21]:
a = x.numpy()
b = torch.tensor(a)
type(a), type(b)

(numpy.ndarray, torch.Tensor)

Converting to a NumPy ndarray, or vice versa, is easy. The converted result does not share memory.

In [22]:
a = x.numpy()
b = torch.tensor(a)
id(b) == id(a)

False

In [23]:
a = x.numpy()
before_a = id(a)
a = torch.tensor(a)
id(a) == before_a

False

To convert a size-one ndarray to a Python scalar, we can invoke the item function or Python’s built-in functions.

In [24]:
a = torch.tensor([3.5])
a, a.item(), float(a), int(a)

(tensor([3.5000]), 3.5, 3.5, 3)

In [25]:
x = torch.arange(12, dtype=torch.float32).reshape((3,4))
y = torch.tensor([[2.0, 1, 4, 3], [1, 2, 3, 4], [4, 3, 2, 1]])
x,y,x == y,x < y,x > y

(tensor([[ 0.,  1.,  2.,  3.],
         [ 4.,  5.,  6.,  7.],
         [ 8.,  9., 10., 11.]]),
 tensor([[2., 1., 4., 3.],
         [1., 2., 3., 4.],
         [4., 3., 2., 1.]]),
 tensor([[False,  True, False,  True],
         [False, False, False, False],
         [False, False, False, False]]),
 tensor([[ True, False,  True, False],
         [False, False, False, False],
         [False, False, False, False]]),
 tensor([[False, False, False, False],
         [ True,  True,  True,  True],
         [ True,  True,  True,  True]]))

In [26]:
a = torch.arange(1, 6, dtype =torch.float32).reshape((5, 1))
b = torch.arange(1, 3).reshape((1, 2))
a, b



(tensor([[1.],
         [2.],
         [3.],
         [4.],
         [5.]]),
 tensor([[1, 2]]))

In [27]:
a + b

tensor([[2., 3.],
        [3., 4.],
        [4., 5.],
        [5., 6.],
        [6., 7.]])

In [28]:
a - b


tensor([[ 0., -1.],
        [ 1.,  0.],
        [ 2.,  1.],
        [ 3.,  2.],
        [ 4.,  3.]])

In [29]:
a * b

tensor([[ 1.,  2.],
        [ 2.,  4.],
        [ 3.,  6.],
        [ 4.,  8.],
        [ 5., 10.]])

In [30]:
a / b

tensor([[1.0000, 0.5000],
        [2.0000, 1.0000],
        [3.0000, 1.5000],
        [4.0000, 2.0000],
        [5.0000, 2.5000]])

In [31]:
a // b

tensor([[1., 0.],
        [2., 1.],
        [3., 1.],
        [4., 2.],
        [5., 2.]])

In [32]:
a \ b

SyntaxError: unexpected character after line continuation character (<ipython-input-32-891beea76adb>, line 1)

In [33]:
a ** b

tensor([[ 1.,  1.],
        [ 2.,  4.],
        [ 3.,  9.],
        [ 4., 16.],
        [ 5., 25.]])

In [34]:
a % b

tensor([[0., 1.],
        [0., 0.],
        [0., 1.],
        [0., 0.],
        [0., 1.]])